<a href="https://colab.research.google.com/github/GirolamoOddo/AppliedMath_Notebooks/blob/main/2D_Solvers_for_HeatEquation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0) _INTRODUCTION TO 2D HEAT EQUATION_  
The 2D heat equation is a fundamental partial differential equation that describes how the temperature in a two-dimensional region changes over time due to heat diffusion. It's commonly used to model various physical phenomena involving heat transfer, such as the cooling of objects, thermal conduction in materials, and temperature distributions in engineering and environmental systems.

__Mathematical Formulation:__    
The 2D heat equation is typically expressed as:

```
dT_dt = alpha * (d2T_dx2 + d2T_dy2)
```
Where:

T(x, y, t) is the temperature distribution in the 2D domain at position (x, y) and time t.
alpha is the thermal diffusivity, a material property that represents how quickly heat diffuses through the medium.
dT_dt represents the rate of change of temperature with respect to time.
d2T_dx2 and d2T_dy2 represent the second derivatives of temperature with respect to the spatial coordinates x and y, respectively. These terms describe the spatial variations in temperature in the x and y directions.

__Physical Interpretation:__  
The 2D heat equation describes how the temperature at any point in the domain changes over time. It states that the rate of change of temperature at a given point is proportional to the Laplacian of the temperature distribution, which represents the spatial variation of temperature in all directions.

---

In [43]:
# @title #####__IMPORT__
locals().clear()
globals().clear()

import numpy as np
import plotly.graph_objects as go
from scipy.special import roots_legendre

#_______________________________________________________________________________
# CHECK PACKAGES VERSION:
#_______________________________________________________________________________
#import os
#packages = [
#    'numpy',
#    'plotly',
#    'scipy',
#]
#
#for package in packages:
#    try:
#        version = os.popen(f"pip show {package} | grep Version").read().strip().split(': ')[1]
#        print(f"{package}: {version}")
#    except Exception as e:
#        print(f"Error getting version for {package}: {e}")
#
#_______________________________________________________________________________
# THE FOLLOWING CODE WORKS PROPERLY WITH THESE PACKAGES VERSION:
#_______________________________________________________________________________
#numpy: 1.25.2
#plotly: 5.15.0
#scipy: 1.11.4
#_______________________________________________________________________________

# 1) _FINITE DIFFERENCE METHOD_

__Space Discretization:__    
The finite difference method discretizes the spatial domain into a grid and approximates spatial derivatives using finite differences. The central difference scheme is employed here for accuracy.  

```
dT_dx2 = (Tn[2:, 1:-1] - 2 * Tn[1:-1, 1:-1] + Tn[:-2, 1:-1]) / dx**2
dT_dy2 = (Tn[1:-1, 2:] - 2 * Tn[1:-1, 1:-1] + Tn[1:-1, :-2]) / dy**2

```  
__Time Discretization:__    
The temporal evolution of temperature is handled through explicit time integration using the forward Euler method.

```
T[1:-1, 1:-1] = Tn[1:-1, 1:-1] + dt * alpha * (dT_dx2 + dT_dy2)

```



In [44]:
# @title #####__Finite Difference Implementation__
# Finite difference scheme
def solve_heat_equation_FD(T0, nt, dt, dx, dy, alpha):
    T = T0.copy()
    for n in range(nt):
        Tn = T.copy()
        T[1:-1, 1:-1] = (Tn[1:-1, 1:-1] + alpha * dt * (
            (Tn[2:, 1:-1] - 2 * Tn[1:-1, 1:-1] + Tn[:-2, 1:-1]) / dx**2 +
            (Tn[1:-1, 2:] - 2 * Tn[1:-1, 1:-1] + Tn[1:-1, :-2]) / dy**2))
    return T


# 2) _FINITE VOLUME METHOD_


__Space Discretization:__    
Finite volume methods divide the domain into control volumes and calculate fluxes at each face of these volumes. In this code, a structured grid is used, and each cell represents a control volume. Heat fluxes are computed at each control volume face based on the temperature gradients.

```
qx[i, j] = -alpha * (Tn[i, j] - Tn[i - 1, j]) / dx
qy[i, j] = -alpha * (Tn[i, j] - Tn[i, j - 1]) / dy

dTxdx = (qx[i, j] - qx[i + 1, j]) / dx
dTydy = (qy[i, j] - qy[i, j + 1]) / dy
```

__Time Discretization:__    
The temporal evolution of temperature is handled by iterating over each control volume and updating the temperature within each volume. The update is based on the fluxes calculated at the faces of each control volume using explicit Euler method.   


```
T[i, j] += dt * (dTxdx + dTydy)

```



In [45]:
# @title #####__Finite Volume Implementation__

# Finite volume scheme

def solve_heat_equation_FV(T0, nt, dt, dx, dy, alpha):
    nx, ny = T0.shape
    T = T0.copy()

    # Define the heat fluxes in the x and y directions
    qx = np.zeros((nx, ny))
    qy = np.zeros((nx, ny))

    for n in range(nt):
        Tn = T.copy()

        # Compute heat fluxes at each control volume face
        for i in range(1, nx - 1):
            for j in range(1, ny - 1):
                qx[i, j] = -alpha * (Tn[i, j] - Tn[i - 1, j]) / dx
                qy[i, j] = -alpha * (Tn[i, j] - Tn[i, j - 1]) / dy

        # Update temperatures within control volumes
        for i in range(1, nx - 1):
            for j in range(1, ny - 1):
                dTxdx = (qx[i, j] - qx[i + 1, j]) / dx
                dTydy = (qy[i, j] - qy[i, j + 1]) / dy
                T[i, j] += dt * (dTxdx + dTydy)

    return T


# 3) _FINITE ELEMENT METHOD_

__Gauss-Lobatto Nodes and Weights:__    
Gauss-Lobatto nodes and weights are computed to accurately integrate over the domain. These nodes are obtained from the roots of the Legendre polynomial of the specified order. The nodes are then rescaled to the domain [0, 1], and equal weights are assigned for integration.

__Basis Functions:__    
Basis functions represent the shape of the element and are used to interpolate the temperature within each element. In this code, basis functions are computed at the Gauss-Lobatto nodes. The basis functions are defined using the product of linear interpolation functions associated with each node.



```
def basis_functions(xi, order=order):
    phi = np.zeros(order + 1)
    for i in range(order + 1):
        phi[i] = np.prod([(xi - nodes[j]) / (nodes[i] - nodes[j]) for j in range(order + 1) if j != i])
    return phi

phi_i = basis_functions(nodes[k])
phi_j = basis_functions(nodes[l])
```



__Finite Element Scheme:__     
The FE scheme employs 5th order basis functions to approximate the temperature distribution. The temperature evolution is handled by iterating over each element. Within each element, the temperature is updated based on the contributions from neighboring elements and the given boundary conditions. Integration is performed using Gauss-Lobatto nodes and weights.

```
rhs +=  dt * alpha * ((phi_i[k] * phi_j[l]) * (Tn[i+1, j] - 2*Tn[i, j] + Tn[i-1, j]) / dx**2
                    + (phi_i[k] * phi_j[l]) * (Tn[i, j+1] - 2*Tn[i, j] + Tn[i, j-1]) / dy**2)
T[i, j] = Tn[i, j] + rhs

```



In [48]:
# @title #####__Finite Element Implementation__

order = 5
# Gauss-Lobatto nodes and weights
nodes, _ = roots_legendre(order+1)  # 6th order Legendre polynomial for 5th order basis
nodes = (nodes + 1) / 2  # Rescale nodes to [0, 1]
weights = np.ones_like(nodes) / 3  # Equal weights for Gauss-Lobatto

# Compute basis functions at Gauss-Lobatto nodes
def basis_functions(xi, order=order):
    phi = np.zeros(order + 1)
    for i in range(order + 1):
        phi[i] = np.prod([(xi - nodes[j]) / (nodes[i] - nodes[j]) for j in range(order + 1) if j != i])
    return phi

# Finite element scheme using 5th order basis functions
def solve_heat_equation_FE(T0, nt, dt, dx, dy, alpha):
    T = T0.copy()

    for n in range(nt):
        Tn = T.copy()
        for i in range(1, nx):
            for j in range(1, ny):
                rhs = np.zeros_like(T[i, j])
                for k in range(order+1):  # Sum over Gauss-Lobatto nodes
                    for l in range(order+1):
                        phi_i = basis_functions(nodes[k])
                        phi_j = basis_functions(nodes[l])
                        rhs +=  dt * alpha * ((phi_i[k] * phi_j[l]) * (Tn[i+1, j] - 2*Tn[i, j] + Tn[i-1, j]) / dx**2
                                            + (phi_i[k] * phi_j[l]) * (Tn[i, j+1] - 2*Tn[i, j] + Tn[i, j-1]) / dy**2)
                T[i, j] = Tn[i, j] + rhs

    return T


# DEBUG CODE____________________________________________________________________
#
# Parameters
#Lx = Ly = 1.0  # Domain size
#nx = ny = 8   # Number of elements in each dimension
#dx = Lx / nx   # Element size in x-direction
#dy = Ly / ny   # Element size in y-direction
#nt = 700       # Number of time steps
#dt = 0.001     # Time step size
#alpha = 0.01   # Thermal diffusivity


#T_fe5 = solve_heat_equation_FE(T0, nt, dt, dx, dy, alpha)

#x_centers = np.linspace(0, Lx, nx+1)
#y_centers = np.linspace(0, Ly, ny+1)
#X, Y = np.meshgrid(x_centers, y_centers)

#fig = go.Figure(data=[go.Surface(z=T_fe5.T, x=X, y=Y)])
#fig.update_layout(title='2D Heat Equation - Finite Element Method with 5th Order Basis',
#                  scene=dict(xaxis=dict(title='X'), yaxis=dict(title='Y'), zaxis=dict(title='Temperature')))
#fig.show()


# 4) _METHODS COMPARISON_

This section compares the solutions of the 2D heat equation using three different numerical methods: Finite Difference (FD), Finite Volume (FV), and Finite Element (FE).

__Parameters:__  
Domain Size: Lx = Ly = 1.0 units   
Grid Resolution: nx = ny = 5 elements  
Time Steps: nt = 5000  
Time Step Size: dt = 0.001 units  
Thermal Diffusivity: alpha = 0.01  

__Initial Condition:__   
Temperature initialized to 100 within a specified region.

__Methods:__  
Solutions obtained using FD, FV, and FE methods
Temperature profiles extracted along y-axis at x = Lx / 2.  

__Visualization:__  
Profiles plotted using Plotly as line plots
Surface plots for initial condition and solutions from each method displayed in 3D.

In [49]:
# @title #####__METHODS COMPARISON__


# Parameters
Lx = Ly = 1.0  # Domain size
nx = ny = 5    # Number of elements in each dimension
dx = Lx / nx   # Element size in x-direction
dy = Ly / ny   # Element size in y-direction
nt = 5000      # Number of time steps
dt = 0.001     # Time step size
alpha = 0.01   # Thermal diffusivity

# Initial condition
T0 = np.zeros((nx+1, ny+1))
T0[int(0.4 + 0.85 / dx):int(1 / dx + 1), int(0.4 + 0.85 / dy):int(1 / dy + 1)] = 100

# Solve the heat equation using differnt methods
T_fd  = solve_heat_equation_FD(T0, nt, dt, dx, dy, alpha)
T_fv  = solve_heat_equation_FV(T0, nt, dt, dx, dy, alpha)
T_fe  = solve_heat_equation_FE(T0, nt, dt, dx, dy, alpha)

profile_fd  = T_fd[:,  int(np.ceil(ny/2))]
profile_fv  = T_fv[:,  int(np.ceil(ny/2))]
profile_fe  = T_fe[:,  int(np.ceil(ny/2))]

x_nodes = np.linspace(0, Lx, nx+1)
y_nodes = np.linspace(0, Ly, ny+1)
X, Y = np.meshgrid(x_nodes, y_nodes)

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=np.linspace(0, Ly, ny+1), y=profile_fd,  mode='lines', name='FD', line=dict(color='blue',  dash='dash')))
fig1.add_trace(go.Scatter(x=np.linspace(0, Ly, ny+1), y=profile_fv,  mode='lines', name='FV', line=dict(color='green', dash='dash')))
fig1.add_trace(go.Scatter(x=np.linspace(0, Ly, ny+1), y=profile_fe,  mode='lines', name='FE', line=dict(color='red',   dash='dash')))

x_nodes = np.linspace(0, Lx, nx+1)
y_nodes = np.linspace(0, Ly, ny+1)
X, Y = np.meshgrid(x_nodes, y_nodes)

fig1.show()

fig2 = go.Figure(data=[go.Surface(z=T0, x=X, y=Y)])
fig2.update_layout(title='2D Heat Equation - IC',
                  scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Temperature'))
fig2.show()

fig3 = go.Figure(data=[go.Surface(z=T_fd, x=X, y=Y)])
fig3.update_layout(title='2D Heat Equation - Finite Difference',
                  scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Temperature'))
fig3.show()

fig4 = go.Figure(data=[go.Surface(z=T_fv, x=X, y=Y)])
fig4.update_layout(title='2D Heat Equation - Finite Volume',
                  scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Temperature'))
fig4.show()

fig5 = go.Figure(data=[go.Surface(z=T_fe, x=X, y=Y)])
fig5.update_layout(title='2D Heat Equation - Finite Element',
                  scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Temperature'))
fig5.show()




---

